# Прогнозирование отправки вагонов в ремонт

In [1]:
import pandas as pd
import numpy as np


In [2]:
# чтобы добраьтся до папки helpers, в которой лежит metrics_f1.py
from os import getcwd
import sys
sys.path.append('\\'.join(getcwd().split('\\')[:-1]))
from helpers.metrics_f1 import calc_f1_score

## Загрузка данных

In [3]:
path_train = r"../data/train_1"
# path_train_2 = r"../data/train_2"

In [6]:
# таргет по прогнозам выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
target['month'] = pd.to_datetime(target['month'])  # преобразование в datetime
target.head()

,wagnum,month,target_month,target_day
0,33361,2023-01-01,0,0
1,33364,2023-01-01,0,0
2,33366,2023-01-01,0,0
3,33358,2023-01-01,0,0
4,33349,2023-01-01,0,0


In [7]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203853 entries, 0 to 203852
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   wagnum        203853 non-null  Int64         
 1   month         203853 non-null  datetime64[ns]
 2   target_month  203853 non-null  Int64         
 3   target_day    203853 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1)
memory usage: 6.8 MB


In [9]:
# какой временной промежуток
target['month'].min(), target['month'].max(), (target['month'].max() - target['month'].min())

(Timestamp('2022-08-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timedelta('153 days 00:00:00'))

In [10]:
# разбиваем данные на train и test
date_to_predict = pd.to_datetime('2023-01-01')
y_train = target[target['month'] < date_to_predict]
y_test = target[target['month'] >= date_to_predict]

In [13]:
# соотноешение train и test
len(y_train), len(y_test), len(y_train) / len(target), len(y_test) / len(target)

(169880, 33973, 0.8333455970724002, 0.16665440292759978)

## Изучение влияния параметров на отправку в ремонт (EDA)

Парметры, которые были выдвинуты в результаты мозгового штурма. Опирались на экспертность и здравый смысл. 

**Технические параметры**
* Модель тележки
* Поглощающий аппарат
* Тип вагона (полувагон, хоппер...)
* Производитель вагона (должен быть какой-то относительный прризнак: ведь чем больше произвоишь, тем больше вагонов и ломается)
* Грузоподъёмность вагона

**Эксплуатационные параметры**
* Возраст вагона
* Общий пробег вагона
* Количество дней с последнего ремонта
* Время езды гружёным / порожним
* Нагрузка при эксплуатации (срадняя, медианная, максимальная)
* Количество ремонтов по каждому виду поломки
* Пробег с последнего ремонта
* Толщина обода колеса (средняя по всем колёсам)
* Высота гребня колеса (средняя по всем колёсам)
* Тип вагона: свой, арендованный...
* Тип погрузки

**Климатические (тоже эксплуатационные)**
* Близость к морю
* Температура и влажность при эксплуатации вагона

### Технические параметры

In [14]:
# Данные по характеристикам вагона
wag_params = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
wag_params.head()

,wagnum,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,zavod_build,date_iskl,cnsi_probeg_dr,cnsi_probeg_kr,kuzov,telega,tormoz,tipvozd,tippogl,norma_km,ownertype
3218,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,2023-02-16,160,160,2,9,3,6,11,110000,0
19128,28344,12-132,1,700,700,88.0,240,2003-08-12,2024-12-24,0,2022-12-14,110,160,2,9,2,1,12,0,0
21526,8099,11-286,0,670,670,138.0,270,1995-08-31,2027-10-01,1,NaT,110,160,2,9,2,1,1,160000,1
32353,33350,12-9850-02,1,750,750,90.0,248,2014-10-27,2047-02-05,19,NaT,250,500,2,11,2,7,12,250000,1
81,5308,11-276,0,680,680,122.0,260,1995-09-17,2027-09-28,1,NaT,110,160,2,9,2,1,11,160000,1



|Переменная | расшифровка|
-------------------------
|wagnum| Номер вагона|

ownertype| Признак передачи вагона в аренду (1-собст, 2-арендованный, 3-инвентарный)
model| модель вагона. Соответствует справочнику Классификатор моделей грузовых вагонов
rod_id| Тип РПС
gruz| грузоподъемность (в центнерах)
cnsi_gruz_capacity| Предельная грузоподъёмность
cnsi_volumek| Объём кузова
tara| Масса тары в центнерах
date_build| дата постройки
srok_sl| дата окончания срока службы
zavod_build| завод постройки. Соответствует справочнику условных кодов предприятий IC00.VAG_UKP
cnsi_probeg_dr| Норма пробега после ДР в тыс. км
cnsi_probeg_kr| Норма пробега после КР в тыс. км
kuzov| материал кузова
telega| код модели тележки
tormoz| Тип тормоза
tipvozd| Тип воздухораспределителя
tippogl| Тип поглощающего аппарата
norma_km| Межремонтный норматив пробега (<> 0, если вагон на пробеге)
new_wagnum| Новый номер вагона
oldnum| Старый номер вагона


In [8]:
# датафрейм с техническими характеристиками вагона и таргетами
df_tech = pd.merge(left=wag_params, right=target, how='inner', 
                   left_on='wagnum', right_on='wagnum')
df_tech.head()

,wagnum,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,zavod_build,...,telega,tormoz,tipvozd,tippogl,norma_km,ownertype,month,target_month,target_day,month_dt
0,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,...,9,3,6,11,110000,0,2022-08-01,1,0,2022-08-01
1,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,...,9,3,6,11,110000,0,2022-09-01,0,0,2022-09-01
2,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,...,9,3,6,11,110000,0,2022-10-01,0,0,2022-10-01
3,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,...,9,3,6,11,110000,0,2022-11-01,0,0,2022-11-01
4,26318,12-600-04,1,682,682,85.0,240,1992-12-25,2022-04-27,5,...,9,3,6,11,110000,0,2022-12-01,0,0,2022-12-01


In [11]:
wag_params.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33977 entries, 3218 to 33707
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   wagnum              33977 non-null  Int64         
 1   model               33977 non-null  string        
 2   rod_id              33977 non-null  Int64         
 3   gruz                33977 non-null  Int64         
 4   cnsi_gruz_capacity  33977 non-null  Int64         
 5   cnsi_volumek        33977 non-null  float64       
 6   tara                33977 non-null  Int64         
 7   date_build          33977 non-null  datetime64[ns]
 8   srok_sl             33977 non-null  datetime64[ns]
 9   zavod_build         33977 non-null  Int64         
 10  date_iskl           116 non-null    datetime64[ns]
 11  cnsi_probeg_dr      33977 non-null  Int64         
 12  cnsi_probeg_kr      33977 non-null  Int64         
 13  kuzov               33977 non-null  Int64  

In [13]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203853 entries, 0 to 203852
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   wagnum        203853 non-null  Int64         
 1   month         203853 non-null  string        
 2   target_month  203853 non-null  Int64         
 3   target_day    203853 non-null  Int64         
 4   month_dt      203853 non-null  datetime64[ns]
dtypes: Int64(3), datetime64[ns](1), string(1)
memory usage: 8.4 MB


In [10]:
df_tech.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203853 entries, 0 to 203852
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   wagnum              203853 non-null  Int64         
 1   model               203853 non-null  string        
 2   rod_id              203853 non-null  Int64         
 3   gruz                203853 non-null  Int64         
 4   cnsi_gruz_capacity  203853 non-null  Int64         
 5   cnsi_volumek        203853 non-null  float64       
 6   tara                203853 non-null  Int64         
 7   date_build          203853 non-null  datetime64[ns]
 8   srok_sl             203853 non-null  datetime64[ns]
 9   zavod_build         203853 non-null  Int64         
 10  date_iskl           694 non-null     datetime64[ns]
 11  cnsi_probeg_dr      203853 non-null  Int64         
 12  cnsi_probeg_kr      203853 non-null  Int64         
 13  kuzov               203853 no

####  Модель вагона

`model`

In [9]:
wagon_model = df_tech.groupby(by='model').aggregate({
    'target_month': ['sum', 'count'],
    'target_day': ['sum', 'count']
}).reset_index()

# более понятные заголовки таблицы, избавляемся от MiltiIndex
wagon_model.columns = wagon_model.columns.map('_'.join)

In [ ]:
# расчёт относительной частоты отправки в ремонт
wagon_model['monthly_freq'] = wagon_model['target_month_sum'] / wagon_model['target_month_count']
wagon_model['daily_freq'] = wagon_model['target_day_sum'] / wagon_model['target_day_count']

In [ ]:
# топ-15 моделей по частоте отправке на ремонт на месячном горизонте
top15_month = wagon_model.sort_values(by='monthly_freq', ascending=False)\
    [['model_', 'monthly_freq', 'target_month_count']].head(15)
top15_month

In [ ]:
# топ-15 моделей по частоте отправке на ремонт на 10-дневном горизонте
top15_10day = wagon_model.sort_values(by='daily_freq', ascending=False)\
    [['model_', 'daily_freq', 'target_day_count']].head(15)
top15_10day

In [ ]:
# Есть ли общие лидеры и по 10 дневным и месячным - да
set.intersection(set(top15_10day['model_']), set(top15_month['model_']))

**Выводы**
1. Модель вагона может оказаться важным фактором
2. Модель может быть использована в качестве метафактора для получения другого фактора - относительная частота ремонтов для модели вагона

#### Тип вагона

In [ ]:
# какие есть модели вагонов
df_tech['model'].unique()

Из моделей можно вытащить тип. Есть система маркировки моделей:
* `11` - крытый вагон
* `12` - полувагон
* `Р-..` - крытый вагон

https://vagon.by/railcars/list/

In [ ]:
df_tech['type'] = df_tech['model'].apply(lambda x: x[:2])
df_tech['type_readable'] = df_tech['type'].apply(lambda x: 'Полувагон' if x == '12' else 'Крытый')

In [ ]:
wagon_type = df_tech.groupby(by='type_readable').aggregate({
    'target_month': ['sum', 'count'],
    'target_day': ['sum', 'count']
}).reset_index()

# более понятные заголовки таблицы, избавляемся от MiltiIndex
wagon_type.columns = wagon_type.columns.map('_'.join)

# расчёт относительной частоты отправки в ремонт
wagon_type['monthly_freq'] = wagon_type['target_month_sum'] / wagon_type['target_month_count']
wagon_type['daily_freq'] = wagon_type['target_day_sum'] / wagon_type['target_day_count']

In [ ]:
wagon_type

**Выводы**
1. Тип вагона, добытый из номера модели, может быть хорошим фактором: полувагоны отправляются на ремонт гораздо чаще крытых вагонов.  

####  Модель тележки

####  Грузоподъёмность вагона

#### Поглощающий аппарат

#### Материал кузова

####  Производитель вагона
Должен быть какой-то относительный прризнак: ведь чем больше произвоишь, тем больше вагонов и ломается)

In [ ]:
%%time

# ---------------------------------- X ----------------------------------------

# данные по дислокации
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()

# данные по текущим ремонтам
pr_rem = pd.read_parquet(path_train + '/pr_rems.parquet').convert_dtypes()

# список вагонов с остаточным пробегом на момент прогноза
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()



# Справочник станций
stations = pd.read_parquet(path_train + '/stations.parquet').convert_dtypes()

# текущие ремонты вагонов
tr_rem = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()

###############################################################################
# --------------------------------- y  ----------------------------------------

In [ ]:
wag_param = wag_param.drop_duplicates(subset='wagnum', keep='last')# у вагонов могут меняться параметры, поэтмоу номер дублируется. В данной модели это фактор не учитывается

In [ ]:
month_to_predict = pd.to_datetime('2022-12-01')

In [ ]:
target.month = pd.to_datetime(target.month)
target = target[target.month == month_to_predict][['wagnum','target_month','target_day']]

In [ ]:
target.target_month.sum(), target.target_day.sum()

# Наивная модель

Наивная модель будет построена на правилах с использованием минимального набора данных, без применения Ml.

Реальный процесс выглядит следующим образом - в начале месяца берется срез по парку по всем вагонам, за ремонт которых несёт ответственность ПГК. Для выбранных вагонов требуется установить, какие из них будут отремонтированы в текущем месяце. Данная информация помогает планировать нагрузку на вагоно-ремонтное предприятие(ВРП). Вторая модель определяет критичные вагоны, которые будут отправлены в ремонт в первую очередь( в ближайшие 10 дней). Это помогает фокусировать внимание диспетчеров.

Основными критериями по которым вагон отправляется в плановый ремонт - является его остаточный пробег и срок до планового ремонта.
В регламентах РЖД используется следующее правило - если ресурс по пробегу не превышает 500 км и/или плановый ремонт должен наступить через 15 дней(или меньше), то вагон может ехать только на ВРП.
Из этого регламента вытекают две особенности:
1. Диспетчер старается отправить вагон раньше положенных значений. Это позволяет выбрать предприятия, на которых ремонтироваться дешевле, а не ближайшее.
2. Компания-оператор может выбирать какому из нормативов нужно следовать - ремонтировать вагон по сроку, или по пробегу, или по обоим критериям сразу. Поэтому встречаются вагоны, у которых пробег может не отслеживаться.

Вагон может быть отправлен в плановый ремонт и раньше положенного. На это может влиять, например, история грузовых операций и количество текущих(мелких) ремонтов. Основная цель участников в данной задаче - найти закономерности и оценить значимые признаки, по которым вагон выбывает в плановый ремонт.

## Подготовка данных

In [ ]:
# оставим только данные по остаточному пробегу для каждого номерав вагона
wag_prob = wag_prob[(wag_prob.repdate == month_to_predict) | (wag_prob.repdate == wag_prob.repdate.min())]

In [ ]:
# оценим среднесуточный пробег из данных по пробегу вагона, на тот случай, если данных по нормативу нет
wag_prob_ =wag_prob.groupby('wagnum', as_index = False).agg({'repdate':['max', 'min'] , 'ost_prob': ['max','min']},)#.droplevel(1)
wag_prob_.columns = [head+'_' + name
                     if head!='wagnum'
                     else head
                     for head, name in wag_prob_.columns ]

wag_prob_['diff_days'] = wag_prob_.repdate_max - wag_prob_.repdate_min
wag_prob_['mean_run'] = (wag_prob_.ost_prob_max - wag_prob_.ost_prob_min )/ wag_prob_.diff_days.dt.days
wag_prob = wag_prob[wag_prob.repdate == wag_prob.repdate.max()][['wagnum','ost_prob']]
wag_prob = wag_prob.merge(wag_prob_[['wagnum','mean_run']])

In [ ]:
# для каждого вагона оставим только информацию по сроку службы и нормативу суточного пробега между ПР
wag_param = wag_param[['wagnum','srok_sl','cnsi_probeg_dr','cnsi_probeg_kr']]

In [ ]:
# добавим признак, что вагон был в ПР в предыдущем месяце. Скорее всего, если вагон был в ПР недавно, то повторно он не поедет
pr_rem['was_repair_in_prev_month'] = 1
pr_rem = pr_rem[['wagnum','was_repair_in_prev_month']]
pr_rem = pr_rem.drop_duplicates(subset='wagnum') #некоторые вагоны все же ремонтируются больше 1 раза, поэтому нужен сбросить дубли

In [ ]:
# посчитаем сколько текущих ремонтов было за прошедший период
tr_rem = tr_rem.groupby('wagnum', as_index= False).kod_vrab.count()

In [ ]:
# сохраним только дату следующего планового ремонта для вагона
dislok = dislok[['wagnum','date_pl_rem']].drop_duplicates(subset = 'wagnum', keep='last')

In [ ]:
# соберем все данные вместе
wp = target[['wagnum']].merge(wag_param, on ='wagnum', how = 'left')\
             .merge(wag_prob, how = 'left')\
             .merge(pr_rem, how = 'left')\
             .merge(tr_rem, how = 'left')\
             .merge(dislok, how = 'left')

In [ ]:
wp.head()

In [ ]:
# Получим среднесуточный пробег, как среднее от нормативов и реального пробега
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp.apply(lambda x : [ val  for val in [x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run] if val != 0], axis = 1 )
wp['day_run']= wp.apply(lambda x : np.mean(x.day_run) if len(x.day_run)> 0 else 0, axis = 1 )

In [ ]:
wp['current_date'] = month_to_predict

In [ ]:
# определим, сколько дней осталось до истечения срока службы
wp['date_diff_srk_sl'] = wp['srok_sl']- wp['current_date']

In [ ]:
# определим, сколько дней осталось до ближайшего ПР
wp['date_diff_pl_rem'] = wp['date_pl_rem']- wp['current_date']

In [ ]:
# определим, какой остаточный ресурс будет на момент окончания месяца
wp['prob_end_month'] = wp['ost_prob'] - wp['day_run']* 30

In [ ]:
wp['target_month'] = 0

In [ ]:
# вагон выбывает в ПР в следующем месяце, если:
# остаточный пробег < 5 000 км
# срок службы < 500 лней
# до следующего  ПР < 40 дней
# ,число текущих ремонтов > 5
wp.loc[(wp.prob_end_month <= 5000) \
       | (wp.date_diff_srk_sl < pd.to_timedelta(40))\
        | (wp.date_diff_pl_rem < pd.to_timedelta(10)) \
        | (wp.kod_vrab > 5),'target_month'] = 1

In [ ]:
wp['target_day'] = wp['target_month']

In [ ]:
pred_target = target[['wagnum']].merge(wp[['wagnum','target_month','target_day']],how = 'left')
pred_target = pred_target.drop_duplicates(subset = 'wagnum')

In [ ]:
# Проверим соотношение отмеченных вагонов с фактическим значением
round(pred_target.target_month.sum() / target.target_month.sum(), 2)

In [ ]:
# сохраним таргет за месяц  для выбранного периода отдельно
target_path = r'../data/train_1/prediction/target_predicton.csv'

In [ ]:
pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

In [ ]:
# путь к файлу с истинными значениями целевой переменной
true_target_path = '../data/train_1/prediction/target_predicton_true.csv'

In [ ]:
target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

In [ ]:
# оценим насколько хорошо удалось предсказать выбытие вагонов  по месяцу и по 10 дням
calc_f1_score(true_target_path, target_path,)